<a href="https://colab.research.google.com/github/yshahanov/deep-convolutional-networks/blob/main/PyTorch_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:


import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms




In [8]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

In [9]:
class DNN(nn.Module):
  def __init__(self):
    super(DNN, self).__init__()
    self.flatten = nn.Flatten()
    self.fc1 = nn.Linear(28 * 28, 256)
    self.bn1 = nn.BatchNorm1d(256)
    self.relu1 = nn.ReLU()
    self.fc2 = nn.Linear(256, 128)
    self.bn2 = nn.BatchNorm1d(128)
    self.relu2 = nn.ReLU()
    self.fc3 = nn.Linear(128, 10)


  def forward(self, x):
    x = self.flatten(x)
    x = self.fc1(x)
    x = self.bn1(x)
    x = self.relu1(x)
    x = self.fc2(x)
    x = self.bn2(x)
    x = self.relu2(x)
    x = self.fc3(x)
    return x


In [10]:
device = "cuda" if torch.cuda.is_available() else "cpu"


In [13]:

model = DNN().to(device)
optimizer = optim.Adam(model.parameters(), lr = 0.01)
criterion = nn.CrossEntropyLoss()

epochs = 5


model.train()
for epoch in range(epochs):
  for inputs, labels in train_loader:
    inputs, labels = inputs.to(device), labels.to(device)
    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

  print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item()}")



model.eval()


correct = 0
total = 0


with torch.no_grad():
  for inputs, labels in test_loader:
    inputs, labels = inputs.to(device), labels.to(device)
    outputs = model(inputs)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()


accuracy = correct / total

print(f'Test Accuracy with Batch Normalization: {accuracy * 100:.2f}%')


Epoch [1/5], Loss: 0.24424462020397186
Epoch [2/5], Loss: 0.1264934092760086
Epoch [3/5], Loss: 0.03444528952240944
Epoch [4/5], Loss: 0.08189254999160767
Epoch [5/5], Loss: 0.07677709311246872
Test Accuracy with Batch Normalization: 97.94%
